In [1]:
import sys
sys.path.append("../")

from SP.stride_prefetch import StridePrefetcherGroup
from IMP.imp import IMPGroup
from DVR.DVR import DVR
from NVR.NVR import NVR



if __name__ == "__main__":
    sp = StridePrefetcherGroup(num_prefetchers=16, default_stride=4)
    imp = IMPGroup(num_units=16)
    dvr = DVR(vector_size=16, ewma_alpha=0.3, initial_bound=2)
    nvr = NVR()
    
    mvin_instructions = [
        [1000, 1004, 1008, 1012, 1016, 1020, 1024, 1028, 1032, 1036, 1040, 1044, 1048, 1052, 1056, 1060], # 第一轮
        [1064, 1068, 1072, 1076, 1080, 1084, 1088, 1092, 1096, 1100, 1104, 1108, 1112, 1116, 1120, 1124], # 第二轮
        [1128, 1132, 1136, 1140, 1144, 1148, 1152, 1156, 1160, 1164, 1168, 1172, 1176, 1180, 1184, 1188]  # 第三轮
    ]

    ss_start = [
        [1000, 1004, 1008, 1012, 1016, 1020, 1024, 1028, 1032, 1036, 1040, 1044, 1048, 1052, 1056, 1060], # 第一轮
        [1064, 1068, 1072, 1076, 1080, 1084, 1088, 1092, 1096, 1100, 1104, 1108, 1112, 1116, 1120, 1124], # 第二轮
        [1128, 1132, 1136, 1140, 1144, 1148, 1152, 1156, 1160, 1164, 1168, 1172, 1176, 1180, 1184, 1188]  # 第三轮
    ]

    ss_end = [
        [1001, 1005, 1009, 1013, 1016, 1020, 1024, 1028, 1032, 1036, 1040, 1044, 1048, 1052, 1056, 1060], # 第一轮
        [1064, 1068, 1072, 1076, 1080, 1084, 1088, 1096, 1098, 1101, 1104, 1108, 1112, 1116, 1120, 1124], # 第二轮
        [1128, 1132, 1136, 1140, 1145, 1148, 1155, 1156, 1160, 1164, 1168, 1174, 1176, 1180, 1184, 1188]  # 第三轮
    ]

    # bound = [
    #     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    #     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    #     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # ]

    bound = [[end - start for start, end in zip(start_list, end_list)] for start_list, end_list in zip(ss_start, ss_end)]
    print(bound)
    
    
    tiled_num = 3
    for i in range(tiled_num-1):
        sp_predict_addr  =  sp.prefetch(mvin_instructions[i]) 
        # imp_predict_addr = imp.prefetch() 
        dvr_predict_addr = dvr.prefetch(bound[i], ss_start[i]) 
        nvr_predict_addr = nvr.prefetch(ss_start[i], ss_end[i]) 
        actrual_addr     = mvin_instructions[i+1]

        print(f"sp预测地址{sp_predict_addr}" )
        # print(imp_predict_addr)
        print(f"dvr_predict_addr预测地址{dvr_predict_addr}")
        print(f"nvr_predict_addr预测地址{nvr_predict_addr}")
        print(f"实际地址{actrual_addr    }")

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 4, 2, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0]]
sp预测地址[(0, 1000, 2000, 1000), (1, 1004, 2008, 1004), (2, 1008, 2016, 1008), (3, 1012, 2024, 1012), (4, 1016, 2032, 1016), (5, 1020, 2040, 1020), (6, 1024, 2048, 1024), (7, 1028, 2056, 1028), (8, 1032, 2064, 1032), (9, 1036, 2072, 1036), (10, 1040, 2080, 1040), (11, 1044, 2088, 1044), (12, 1048, 2096, 1048), (13, 1052, 2104, 1052), (14, 1056, 2112, 1056), (15, 1060, 2120, 1060)]
dvr_predict_addr预测地址[(0, 1000), (0, 1001), (1, 1004), (1, 1005), (2, 1008), (2, 1009), (3, 1012), (3, 1013), (4, 1016), (5, 1020), (6, 1024), (7, 1028), (8, 1032), (9, 1036), (10, 1040), (11, 1044), (12, 1048), (13, 1052), (14, 1056), (15, 1060)]
nvr_predict_addr预测地址[(0, 1000), (1, 1004), (2, 1008), (3, 1012)]
实际地址[1064, 1068, 1072, 1076, 1080, 1084, 1088, 1092, 1096, 1100, 1104, 1108, 1112, 1116, 1120, 1124]
sp预测地址[(0, 1064, 1128, 64), (1, 1068, 1132, 64), (2, 10